In [1]:
from backends import ModelSpec, Model, get_model_for, load_model_registry
#need to add api key
#need to add model_name in the dict
#need to create key.json
load_model_registry()  # load default model registry from backends folder
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106") # works without registered entry when openai_api.py backend is available


In [2]:
lmm: Model = get_model_for(THIS_MODEL)

lmm.set_gen_args(temperature=0.0, max_tokens=400)


In [157]:
import re

#system prompt
with open('resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    sys_prompt = f.read()

#initial instruction
with open('resources/initial_prompts/no_board_multitoken.txt', 'r') as f:
    content = f.read()

prefix = "MY MOVE:"
class Game():
    def __init__(self, lmm, initial_msg, sys_prompt):
        self.num_fields = 21
        self.current_pos = 0
        self.msg = initial_msg #initial message -> contains game instructions
        self.last_response = []
        self.sys_prompt = sys_prompt #system prompt
        self.lmm = lmm #language model object
        self.rolls = [3,6,4,5,2,6,4,1] #list of rolls per turn
        self.board = " □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ "
        self.turn = 0 #turn counter
        self.context = [] #memorized context
        self.current_state = '' #current state of the board
        self.prefix = "MY MOVE:" #used to parse the response
        self.sign = " -> "
        self.token_out = False #false if no token on board yet.
        

    def make_move(self):
        if self.turn == 0: #first turn prompt is slightly different
            in_m = f"Are you ready to play?\nBeginning State: {self.board}\n Turn num: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."
            self.add_message(self.msg + '\n' + in_m)
            init_prompt, init_resp, init_resp_text = self.lmm.generate_response(self.context)
            #needs try catch shit here, and everywhere we actually parse/split/index/etc.
            move = self.parse_reply(init_resp_text)
            #check validity of the move
            if self.check_move(move, self.rolls[self.turn]) == True:
                
                print(self.current_state)
                print(move)
                self.current_pos = move 
                self.add_message(init_resp_text, role = 'assistant' ) #if valid, adds response to context
                self.turn+=1 #updates step
                if move == 6:
                    self.update_board(move)
                    self.token_out = True 
                    self.last_response = init_resp_text
            else:
                print(f'fail at turn: {self.turn}')
                print(f'roll: {self.rolls[self.turn]}')
                print(f'{init_resp_text}')
                self.last_response = init_resp_text
                print(self.current_state)
                
        else:
            #if not first turn
            m = f"Current state: {self.current_state}\n Next turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\n Where will you move your token? Let's think step by step."
            self.add_message(m, role = 'user') 
            given_prompt, response, response_text = lmm.generate_response(self.context)
            
            move = self.parse_reply(response_text)
            if  self.check_move(move, self.rolls[self.turn]) == True:
                print(self.current_state)

                if self.token_out == False and self.rolls[self.turn] == 6:
                    self.token_out = True
                       
                self.current_pos = move 
                self.add_message(response_text, role ='assistant')
                self.turn+=1
                
                if self.token_out:
                    self.update_board(move)
                print(self.current_state)

            else: 
                print(f'fail at turn: {self.turn}')
                print(f'roll: {self.rolls[self.turn]}')
                print(f'{response_text}')
                print(f'move: {move}')
                print(self.current_state)
                
    def check_move(self, move, roll):
        #if token not out, true only if rolled 6 and moved to pos 1.
        if self.token_out == False:
            
            if roll == 6 and move == 1:
                return True
            elif roll !=6 and move == 0:
                return True
            else:
                return False
            
        if self.token_out:
            #no overreaching
            if move > self.num_fields:
                print(f'f1')
                return False
            #no backsies
            if self.current_pos > move:
                print('f2')
                return False
            
            if roll + self.current_pos == move:
                return True
            
            #if llm doesn't move when roll > remaining fields, return True.
            elif (roll + self.current_pos > self.num_fields) and self.current_pos == move:
                return True
            else:
                print(f'fails here {roll + self.current_pos} != {move} ')
                return False
            
    
    
    def parse_reply(self, text):
        # parses the reply

        parsed = text.split(self.prefix)
        print(parsed[1])
        final = parsed[1].split(self.sign)
        print(f'final: {final}')
        return int(final[1])
   
        
    def update_board(self, move):
        b = self.board
        split_b = b.split()
        split_b[move-1] = 'X'
        new_b = ' '.join(split_b)
        self.current_state = new_b
    
    
        
    def add_message(self, message, role='user'):
        #add message to context
        if self.context == []:
            self.context = [
                {"role": "system", "content": self.sys_prompt}
            ]
        self.context.append({"role": role, "content": message})

    def reset(self):
        #resets game state
        self.turn = 0
        self.context = []

In [158]:
instance = Game(lmm, content, sys_prompt)

In [159]:
instance.turn

In [174]:
instance.make_move()


In [175]:
instance.context

In [17]:
t = 0

def asd(t):
    if t == 0:
        return 4
    if t == 0:
        return 1
    

In [18]:
v  = asd(t)

In [38]:
' X -> 0'.split(' -> ')

In [24]:
def update_board(move):
    b = '□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □'
    split_b = b.split()
    split_b[move-1] = 'X'
    new_b = ' '.join(split_b)
        
    return new_b
        
    

In [25]:
b = update_board(5)

In [26]:
b